In [1]:
import pandas as pd
import numpy as np
import sys
import pickle
from Levenshtein import distance as lev
from os import listdir

from tools import get_elevation_of_point, processed_path, crawl_path, distance

# Join all meta data to a final product

In [2]:
# path where processed data is saved
path = processed_path

# Used for filtering.
missing_stats = pickle.load(open(path + "missing_stats.p", "rb"))

onlyfiles = [path + f for f in listdir(path) if "meta_data" in f]


meta = {}
for x in onlyfiles:
    meta[x.split("/")[-1].split("_meta_data")[0]] = pd.read_csv(x, dtype=str)

print(meta.keys())

dict_keys(['bscv', 'mv', 'saxony_anhalt', 'thuringia', 'bavaria', 'brandenburg', 'saxony', 'berlin'])


### Final Join.

In [3]:
meta["saxony"]["O"] = "S"
meta["saxony_anhalt"]["O"] = "SA"
meta["bscv"]["O"] = "BSCV"
meta["thuringia"]["O"] = "T"
meta["bavaria"]["O"] = "BA"
meta["brandenburg"]["O"] = "BR"
meta["berlin"]["O"] = "B"
meta["mv"]["O"] = "MV"

In [4]:
meta["bavaria"].rename(columns={"Unnamed: 0": "ID"}, inplace=True)
meta["mv"].rename(columns={"Unnamed: 0": "ID"}, inplace=True)

In [5]:
joint = pd.concat([meta[x] for x in meta.keys()])
joint.reset_index(inplace=True, drop=True)
joint

,ID,R,X,Y,D,H,QD,QH,QX,QY,QR,O
0,570910,Saale,51.796792388633605,11.73542296734891,NaN,55.125,-1,0,0,0,0,BSCV
1,502350,Elbe,52.541306291748874,11.978055933719979,NaN,27.556,-1,0,0,0,0,BSCV
2,501010,Elbe,50.875763038508225,14.235236598702063,NaN,116.2,-1,0,0,0,0,BSCV
3,501420,Elbe,51.85652995414649,12.646310267773282,NaN,62.45,-1,0,0,0,0,BSCV
4,502210,Elbe,52.18095358430071,11.68265360487722,NaN,37.203,-1,0,0,0,0,BSCV
...,...,...,...,...,...,...,...,...,...,...,...,...
1276,5826701,Gosener Kanal,52.40249480445807,13.703589046331425,1.00,31.81,0,0,0,0,0,B
1277,5862811,Zeuthener See,52.37374240071633,13.653000966271387,11.20,31.94,0,0,0,0,0,B
1278,5827700,Spree,52.53616784466021,13.216279031086982,0.60,28.00,0,0,0,0,0,B
1279,5803200,Havel,52.52447996396127,13.205033354431544,1.48,28.01,0,0,0,0,0,B


In [6]:
joint["X"] = joint["X"].astype(float)
joint["Y"] = joint["Y"].astype(float)
joint["D"] = joint["D"].astype(float)
joint["H"] = joint["H"].astype(float)
joint["QX"] = joint["QX"].astype(int)
joint["QY"] = joint["QY"].astype(int)
joint["QD"] = joint["QD"].astype(int)
joint["QH"] = joint["QH"].astype(int)

### Remove duplicates etc.

In [7]:
# Remove bricked data:
print(len(joint))
bricked = pickle.load(open(path + "/bricked_columns.p", "rb"))
print(len(bricked))

1281
79


In [8]:
to_drop = []
for x in bricked:
    try:
        check = joint[(joint["ID"] == x.split("_")[0]) & (joint["O"] ==  x.split("_")[1].upper())].index.values[0]
        to_drop.append(check)
    except:
        print(x,"ISSUE")
# there is no meta data for these. 

572050_t ISSUE
5746020_t ISSUE


In [9]:
print(len(joint))
joint.drop(index=to_drop, inplace=True)
print(len(joint))

1281
1204


In [10]:
doubled = joint["ID"].value_counts().sort_values()

In [11]:
joint[joint["ID"].isin(doubled[doubled > 1].index)].sort_values("ID")

,ID,R,X,Y,D,H,QD,QH,QX,QY,QR,O
109,575710,Thyra,51.448969,11.004660,1.20,152.000,0,0,0,0,0,SA
351,575710,Thyra,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,T
257,576440,Weie Elster,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,T
1132,576440,Weiße Elster,50.610847,12.168172,170.10,271.225,0,0,0,0,0,S
163,576610,Weiße Elster,51.057096,12.143712,89.48,146.440,0,0,0,0,0,SA
313,576610,Weie Elster,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,T
347,577540,Pleie,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,T
1220,577540,Pleiße,51.082791,12.443781,32.60,143.350,0,0,0,0,0,S
261,577901,Whyra,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,T
1252,577901,Wyhra,51.045811,12.569950,21.80,162.380,0,0,0,0,0,S


In [12]:
drop_list = [351,257,313,347,261]

In [13]:
print(missing_stats["5827700" + "_br"],missing_stats["5827700" + "_b"])
drop_list.append(1024)

0.05249406717780212 0.0005419404892296458


In [14]:
print(missing_stats["5862811" + "_br"],missing_stats["5862811" + "_b"])
# b is very noise so keep br
drop_list.append(1277)

0.02390813253012048 0.0009070372398685652


In [15]:
joint = joint.drop(index=drop_list)

In [16]:
joint.index = joint["ID"]
joint.drop(columns=["ID"], inplace=True)

In [17]:
joint.index.value_counts().sort_values()

570910     1
502070     1
603080     1
501480     1
570930     1
          ..
5803200    1
551601     1
576411     1
567700     1
5866301    1
Name: ID, Length: 1197, dtype: int64

### Estimate missing coordinates by hand.

In [18]:
joint.isnull().sum()

R       0
X       9
Y       9
D     610
H     189
QD      0
QH      0
QX      0
QY      0
QR      0
O       0
dtype: int64

In [19]:
# check if we have data for this or generally broken: 
repair = joint[joint["X"].isnull()]
print(len(repair))
repair

9


,R,X,Y,D,H,QD,QH,QX,QY,QR,O
ID,,,,,,,,,,,
5771000,Trieb,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,T
5771200,Rabenbach,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,T
5775010,Pleie,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,T
427000,Ulster,NaN,NaN,30.0,NaN,0,-1,-1,-1,0,T
5764200,Weie Elster,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,T
5771100,Trieb,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,T
5772200,Gltzsch,NaN,NaN,NaN,NaN,-1,-1,-1,-1,0,T
420130,Werra,NaN,NaN,157.0,NaN,0,-1,-1,-1,0,T
427011,Ulster,NaN,NaN,1.3,NaN,0,-1,-1,-1,0,T


In [20]:
joint[joint["R"] == "Pleie"]
joint.drop("5775010",inplace=True) # Same

In [21]:
joint[joint["R"] == "Werra"] # more than enough available
joint.drop("420130", inplace=True) # Multiple available

In [22]:
joint[joint["R"] == "Rabenbach"] # available and probably the same. 
joint.drop("5771200", inplace=True) # Multiple available

In [23]:
(joint[joint["R"] == "Göltzsch"])  # available and probably the same.
(joint[joint["R"] == "Gltzsch"])  # available and probably the same.)
joint.drop("5772200", inplace=True) # Multiple available

In [24]:
print(missing_stats["5771000" + "_t"],missing_stats["5771100" + "_t"])
# not really recovarable. remove one and set the other one to the some coordinates on the river.


joint.loc["5771000", ["X", "Y"]] = 50.51688662693567, 12.234923941657728
joint.loc["5771000", "QX"] = 1
joint.loc["5771000", "QY"] = 1


joint.drop("5771100", inplace=True) # Multiple available

0.2560126871120847 0.2557331599123768


In [25]:
# We can estimate these as we have the distance to the end of the river.
joint.loc["427000", ["X", "Y"]] = 50.63392770634668, 10.014996710046734 # Rough
joint.loc["427011", ["X", "Y"]] = 50.83054339469407, 9.991035269977047
joint.loc["427000", "QX"] = 1
joint.loc["427000", "QY"] = 1
joint.loc["427011", "QX"] = 1
joint.loc["427011", "QY"] = 1

In [26]:
joint[joint["R"] == "Weiße Elster"] # like double id. not necessary
joint.drop("5764200", inplace=True) # Multiple availableb


### Estimate the H based on coordinated where it missing.

In [27]:
joint.isnull().sum()

R       0
X       0
Y       0
D     605
H     183
QD      0
QH      0
QX      0
QY      0
QR      0
O       0
dtype: int64

In [29]:
try: 
    h_meta_map = pickle.load(open(crawl_path + "/meta_h_map.p", "rb"))
    print("load..")
except:
    h_meta_map = {}
for ind, line in joint[joint["H"].isnull()].iterrows():
    if int(ind) in h_meta_map:
        pass
    else:
        try:
            h_meta_map[ind] = get_elevation_of_point([line.X, line.Y])[0]
        except:
            print("Fail: "+ str(ind))
            pass
pickle.dump(h_meta_map,open(crawl_path + "/meta_h_map.p", "wb"))

load..


In [30]:
for ind, line in joint[joint["H"].isnull()].iterrows():
    try:
        joint.loc[ind,"H"] = h_meta_map[int(ind)]
        joint.loc[ind, "QH"] = 2
    except:
        pass

In [32]:
# Positions unknown
joint["H"].isnull().sum()

0

### Reestimate havel as its height is broken. (This might need some further work)

In [33]:
joint[joint["R"] == "Havel"]

,R,X,Y,D,H,QD,QH,QX,QY,QR,O
ID,,,,,,,,,,,
5802600,Havel,52.773140,13.242927,4.40,30.168,0,0,0,0,0,BR
5816802,Havel,52.945889,13.375635,32.10,40.392,0,0,0,0,0,BR
5805200,Havel,52.655951,12.333705,-109.83,24.505,0,0,0,0,0,BR
5804300,Havel,52.463142,12.856784,-34.05,28.386,0,0,0,0,0,BR
5807900,Havel,52.823194,12.076555,-145.26,21.561,0,0,0,0,0,BR
5815901,Havel,52.702783,13.250698,-20.30,28.937,0,0,0,0,0,BR
5803200,Havel,52.524480,13.205033,1.48,28.010,0,0,0,0,0,B


In [34]:
havel_h = {}
for ind, line in joint[joint["R"] == "Havel"].iterrows():
    havel_h[ind] = get_elevation_of_point([line.X, line.Y])[0]

for ind, line in joint[joint["R"] == "Havel"].iterrows():
    joint.loc[ind,"H"] = havel_h[ind]
    joint.loc[ind, "QH"] = 2


In [35]:
joint[joint["R"] == "Havel"].sort_values("H")

,R,X,Y,D,H,QD,QH,QX,QY,QR,O
ID,,,,,,,,,,,
5807900,Havel,52.823194,12.076555,-145.26,24.0,0,2,0,0,0,BR
5805200,Havel,52.655951,12.333705,-109.83,25.0,0,2,0,0,0,BR
5804300,Havel,52.463142,12.856784,-34.05,31.0,0,2,0,0,0,BR
5803200,Havel,52.524480,13.205033,1.48,31.0,0,2,0,0,0,B
5815901,Havel,52.702783,13.250698,-20.30,36.0,0,2,0,0,0,BR
5802600,Havel,52.773140,13.242927,4.40,39.0,0,2,0,0,0,BR
5816802,Havel,52.945889,13.375635,32.10,41.0,0,2,0,0,0,BR


### Custom Fixes

In [36]:
# same station with varying id. one is broken so remove: 

#Mittelwass/stollen#

joint.drop(index = "574670", inplace=True)
#joint.drop(index = "10026302", inplace=True) # double donau station that was somehow not filtered.

### Filter double stations: 

In [37]:
# Calculate all distances
distances = np.array([[distance(y, x) for x in joint[["X", "Y"]].values] for y in joint[["X", "Y"]].values])
namings = list(joint.index)

In [38]:
#select the pairs with very low distance
package = []
for n,x in enumerate(distances): 
    if (x < 0.5).sum() == 1: 
        pass
    else:
        close = [(w, x[w]) for w in np.where(x < 0.5)[0] if w != n]
        for candidate in close:
            package.append([namings[n],namings[candidate[0]], candidate[1]])

In [39]:
remain = []
fine = []
for x in package: 
    # If they come from the same source, probably not double.
    if joint.loc[x[0],"O"] == joint.loc[x[1],"O"]:
        fine.append(x)
    else:
        # if they are from the same river and the id differs only by 1  character, this is very likely the same station.
        if joint.loc[x[0],"R"] == joint.loc[x[1],"R"]:
            if lev(str(x[0]), str(x[1])) == 1: 
                remain.append(x)
            else:   
                fine.append(x)
        else:
            remain.append(x)

In [40]:
# we mean over both if they have data available and take the index with the lower amount of missing data.
# If only one is available we take this one.

to_remove = []
for x in remain: 
    double = joint.loc[x[:2]]
    available = [x for x in missing_stats.index if str(double.index[0]) +"_" in x]
    available2 = [x for x in missing_stats.index if str(double.index[1])+ "_" in x]

    if len(available2) == 2:
        # small fix
        print("correct2")
        available2 = available2[:1]
        print(available2)

    if len(available) == 2:
        # small fix
        print("correct")
        available = available[:1]
        print(available)
    if (len(available) == 1) and (len(available2) == 1):
        available = missing_stats[available[0]]
        available2 = missing_stats[available2[0]] 
        keep = double.index[0] if available > available2 else double.index[1]
        to_remove.append(double.index[1] if available > available2 else double.index[0])
        values = double[["X", "Y", "D", "H"]].mean().values
        joint.loc[keep,["X", "Y", "D", "H"]] = values
        joint.loc[keep,["QD","QH","QX","QY"]] = 3

    elif (len(available) == 1) and (len(available2) == 0):
        to_remove.append(double.index[1])

    elif (len(available) == 0) and (len(available2) == 1):
        to_remove.append(double.index[0]) 

    else:
        print(x, "Fail")


correct2
['660100_s']
correct
['660100_s']


In [41]:
joint = joint.drop(index=to_remove)


In [42]:
joint.isnull().sum()

R       0
X       0
Y       0
D     591
H       0
QD      0
QH      0
QX      0
QY      0
QR      0
O       0
dtype: int64

In [43]:
joint[joint["O"].isnull()]

,R,X,Y,D,H,QD,QH,QX,QY,QR,O
ID,,,,,,,,,,,


In [44]:
joint.index = joint.index.astype(str) + "_" + joint["O"].astype(str).str.lower()

In [46]:
joint.to_csv(path + "meta_joined.csv")